<a href="https://colab.research.google.com/github/hukgithub/AR_AI_Makeup/blob/main/AR_based_Generative_AI_Makeup_and_SkincareConsultant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Explanation of the Code
**Facial Landmark Detection with MediaPipe**:

The apply_virtual_makeup function uses MediaPipe’s Face Mesh model to detect facial landmarks.
A simple blush effect is simulated by drawing blue circles on specific landmark points. This part can be expanded for more makeup effects.
Voice Recognition with Whisper:

The transcribe_voice function converts the user’s audio input into text using the Whisper model.
Skincare Recommendations with Hugging Face:

The generate_skincare_recommendation function takes a user query and generates skincare advice using the distilgpt2 model from Hugging Face. This prompt can be customized to include specific skin concerns.
Web Interface with Gradio:

The Gradio interface sets up a simple web app where users can:
Capture an image via webcam.
Optionally record a voice message for skincare advice.
See the virtual makeup effect on their image and get text-based skincare advice.
Running the App
Run this code in Google Colab.
The Gradio app will launch and give you a shareable link to test the web-based AR experience.
Limitations & Possible Extensions
Makeup Simulation: This example uses basic dots to represent blush. To expand, you could add color blending or more advanced face makeup.
AR on Browser: While browser-based AR is limited compared to mobile ARKit/ARCore, this approach demonstrates key aspects of the concept.
This setup provides a platform-independent web-based experience, allowing users to interact with the skincare consultant app entirely through their browsers! Let me know if you’d like to expand on any of these features.

In [ ]:
!pip install gradio transformers torch openai-whisper mediapipe opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [ ]:
import gradio as gr
from transformers import pipeline
import whisper
import cv2
import numpy as np
import mediapipe as mp
import torch

# Load Whisper model for voice transcription
device = "cuda" if torch.cuda.is_available() else "cpu"
transcribe_model = whisper.load_model("base", device=device)

# Load a pre-trained language model for skincare advice
skincare_model = pipeline("text-generation", model="distilgpt2", device=0 if device == "cuda" else -1)

# Initialize MediaPipe for facial landmark detection
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

def apply_virtual_makeup(image):
    # Detect facial landmarks and apply virtual makeup
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Draw landmarks with makeup effect
            for landmark in face_landmarks.landmark:
                x = int(landmark.x * image.shape[1])
                y = int(landmark.y * image.shape[0])
                cv2.circle(image, (x, y), 1, (255, 0, 0), -1)  # Apply "blush" with blue points as example

    return image

def transcribe_voice(audio):
    transcription = transcribe_model.transcribe(audio)["text"]
    return transcription

def generate_skincare_recommendation(text_input):
    prompt = f"Generate skincare advice for the following concerns: {text_input}"
    response = skincare_model(prompt, max_length=100, num_return_sequences=1)
    return response[0]['generated_text']

# Gradio Interface
def skincare_consultant(input_image, audio_input=None):
    if audio_input:
        question_text = transcribe_voice(audio_input)
    else:
        question_text = "Give general skincare advice."

    skincare_advice = generate_skincare_recommendation(question_text)
    makeup_applied_image = apply_virtual_makeup(input_image)

    return makeup_applied_image, skincare_advice

# Set up Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# AR-Based AI Makeup and Skincare Consultant")

    with gr.Row():
        input_image = gr.Image(type="numpy", label="Upload Face Image")  # Changed to upload instead of webcam capture
        audio_input = gr.Audio(type="filepath", label="Ask a Skincare Question (Optional)")

    makeup_image = gr.Image(label="Virtual Makeup Applied")
    skincare_advice_output = gr.Textbox(label="Skincare Advice")

    submit_button = gr.Button("Get Advice and Apply Makeup")
    submit_button.click(
        skincare_consultant,
        inputs=[input_image, audio_input],
        outputs=[makeup_image, skincare_advice_output]
    )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aa60737e7183c7e648.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
